<a href="https://colab.research.google.com/github/sampathrajapaksha/My-First/blob/master/ktrain_sentence_pair_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import pandas as pd
import nltk
from sklearn.metrics import f1_score,precision_score,recall_score, confusion_matrix,classification_report

Mount google drive

In [ ]:
'''import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.append(nb_path)'''

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/mnt


In [ ]:
!pip install --target=$nb_path simpletransformers

In [149]:
!pip3 install ktrain

In [21]:
# import file from google drive
from google.colab import files
uploaded = files.upload()

Saving Annotation All.xlsx to Annotation All.xlsx


In [38]:
#import data from google drive
import io
data = pd.read_excel(io.BytesIO(uploaded['Annotation All.xlsx']))

In [ ]:
#import data from github
url = 'https://github.com/sampathrajapaksha/My-First/blob/master/Annotation%20All.xlsx?raw=true'
data_1 = pd.read_excel(url)

In [130]:
# replace Nan with 0
columns =['Fit','Size','Material','Comfortability','Quality','Price','Color','General']
for col in columns:
    data[col].fillna(0,inplace=True)

In [132]:
data.index.names=['Index']
data.head()

,Sentence_No,ID,New_ID,Product,Review,Sentence,Fit,Size,Material,Comfortability,Quality,Price,Color,General
Index,,,,,,,,,,,,,,
0,641,50532,50532.0,Nike Mens Club Swoosh Hoodie,My son loved his sweater but it was a bit too ...,My son loved his sweater but it was a bit too ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,642,50532,NaN,NaN,NaN,I would suggest ordering a size up,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,643,46575,46575.0,Mens Nike KO 1/4 Zip Top,Cannot go wrong with Nike products for the mos...,Cannot go wrong with Nike products for the mos...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,644,46575,NaN,NaN,NaN,Fit and finish is excellent.,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,645,46575,NaN,NaN,NaN,Also surprised how water resistant the pullove...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [134]:
# filter data
data_review = data[['Sentence', 'Fit','Size', 'Material', 'Comfortability', 
                    'Quality', 'Price', 'Color',
                    'General']]
data_review['Sentence_len'] = data_review['Sentence'].str.len()

#remove sentence with lenth less than 4 
data_review = data_review[data_review['Sentence_len']>4]
data_review.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Sentence,Fit,Size,Material,Comfortability,Quality,Price,Color,General,Sentence_len
Index,,,,,,,,,,
0,My son loved his sweater but it was a bit too ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,113.0
1,I would suggest ordering a size up,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0
2,Cannot go wrong with Nike products for the mos...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,53.0
3,Fit and finish is excellent.,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0
4,Also surprised how water resistant the pullove...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,51.0


In [135]:
#Sentence tokanize
nltk.download('punkt')
data_review['Sentence']=data_review['Sentence'].str.lower()
nltk.download('punkt')
data_review['Sentence'].dropna(inplace=True)
data_review['review'] = data_review['Sentence'].apply(nltk.word_tokenize)
data_review.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Sentence,Fit,Size,Material,Comfortability,Quality,Price,Color,General,Sentence_len,review
Index,,,,,,,,,,,
0,my son loved his sweater but it was a bit too ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,113.0,"[my, son, loved, his, sweater, but, it, was, a..."
1,i would suggest ordering a size up,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,"[i, would, suggest, ordering, a, size, up]"
2,cannot go wrong with nike products for the mos...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,53.0,"[can, not, go, wrong, with, nike, products, fo..."
3,fit and finish is excellent.,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,"[fit, and, finish, is, excellent, .]"
4,also surprised how water resistant the pullove...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,51.0,"[also, surprised, how, water, resistant, the, ..."


In [136]:
data_review['review_len'] = data_review['review'].apply(len)
print('Min word lenth :',data_review['review_len'].min())
print('Max word lenth :',data_review['review_len'].max())

Min word lenth : 1
Max word lenth : 91


In [137]:
data_review.shape

(10032, 12)

In [138]:
#data for model
import numpy as np
df_all = data_review[['Sentence','Fit','Size', 'Material', 'Comfortability', 'Quality',
       'Price', 'Color', 'General']]
cols = ['Fit','Size', 'Material', 'Comfortability', 'Quality','Price', 'Color', 'General']
df_all[cols] = df_all[cols].applymap(np.int64)
print('full dataframe size: ',df_all.shape)
df_test = df_all[0:2000]
print('test dataframe size: ',df_test.shape)
df_all_train = df_all[2000:]
print('train dataframe size: ',df_all_train.shape)

full dataframe size:  (10032, 9)
test dataframe size:  (2000, 9)
train dataframe size:  (8032, 9)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [140]:
df_all_train.head()

,Sentence,Fit,Size,Material,Comfortability,Quality,Price,Color,General
Index,,,,,,,,,
2010,no fading and everything seems to hold its sha...,0,0,0,0,0,0,0,1
2011,i do recommend.,0,0,0,0,0,0,0,1
2012,very comfortable.,0,0,0,1,0,0,0,0
2013,hold everything in place as expected during al...,0,0,0,0,0,0,0,1
2014,i own 3 of these and love them.,0,0,0,0,0,0,0,1


#ktrain example

In [141]:
# import file from google drive
from google.colab import files
uploaded = files.upload()


In [ ]:
#import data from google drive
import io, csv
data = pd.read_csv(io.BytesIO(uploaded['msr_paraphrase_test.txt']))

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [142]:
import csv
TRAIN = '/content/drive/My Drive/Research/Data/msr_paraphrase_train.txt'
TEST = '/content/drive/My Drive/Research/Data/msr_paraphrase_test.txt'

In [143]:
train.head()

,Quality,#1 ID,#2 ID,#1 String,#2 String
0,1,1089874,1089925,"PCCW 's chief operating officer , Mike Butcher...",Current Chief Operating Officer Mike Butcher a...
1,1,3019446,3019327,The world 's two largest automakers said their...,Domestic sales at both GM and No. 2 Ford Motor...
2,1,1945605,1945824,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...
3,0,1430402,1430329,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...
4,0,3354381,3354396,The company didn 't detail the costs of the re...,But company officials expect the costs of the ...


In [144]:
import csv
#TRAIN = 'msr_paraphrase_train.txt'
#TEST = 'msr_paraphrase_test.txt'
train_df = pd.read_csv(TRAIN, delimiter='\t', quoting=csv.QUOTE_NONE)
test_df = pd.read_csv(TEST, delimiter='\t', quoting=csv.QUOTE_NONE)

x_train = train_df[['#1 String', '#2 String']].values
y_train = train_df['Quality'].values
x_test = test_df[['#1 String', '#2 String']].values
y_test = test_df['Quality'].values


# IMPORTANT: data format for sentence pair classification is list of tuples of form (str, str)
x_train = list(map(tuple, x_train))
x_test = list(map(tuple, x_test))

#x_train

In [145]:
print(x_train[0])
print(y_train[0])

('Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .')
1


In [150]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=128, class_names=['not paraphrase', 'paraphrase'])
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32) # lower bs if OOM occurs
learner.fit_onecycle(5e-5, 3)

preprocessing train...
language: en
sentence pairs detected


Is Multi-Label? False
preprocessing test...
language: en
sentence pairs detected


ValueError: ignored

# Model Buliding

Different dataframs for aspects for binary classification

In [121]:
# Fit
df_fit = df_all_train[['Sentence','Fit']]
df_fit = df_fit.rename(columns={'Fit':'Class'})
df_fit['Aspect'] = 'Fit'

df_test_fit = df_test[['Sentence','Fit']]
df_test_fit = df_test_fit.rename(columns={'Fit':'Class'})
df_test_fit['Aspect'] = 'Fit'


# Size
df_size = df_all_train[['Sentence','Size']]
df_size = df_size.rename(columns={'Size':'Class'})
df_size['Aspect'] = 'Size'

df_test_size = df_test[['Sentence','Size']]
df_test_size = df_test_size.rename(columns={'Size':'Class'})
df_test_size['Aspect'] = 'Size'


# Material
df_material = df_all_train[['Sentence','Material']]
df_material = df_material.rename(columns={'Material':'Class'})
df_material['Aspect'] = 'Material'

df_test_material = df_test[['Sentence','Material']]
df_test_material = df_test_material.rename(columns={'Material':'Class'})
df_test_material['Aspect'] = 'Material'


# Comfortability
df_comfortability = df_all_train[['Sentence','Comfortability']]
df_test_comfortability = df_test[['Sentence','Comfortability']]

# Quality
df_quality = df_all_train[['Sentence','Quality']]
df_test_quality = df_test[['Sentence','Quality']]

# Price
df_price = df_all_train[['Sentence','Price']]
df_test_price = df_test[['Sentence','Price']]

# Color
df_color = df_all_train[['Sentence','Color']]
df_test_color = df_test[['Sentence','Color']]

# General
df_general = df_all_train[['Sentence','General']]
df_test_general = df_test[['Sentence','General']]

In [61]:
df_test_fit.Class.value_counts()

0    1721
1     279
Name: Class, dtype: int64

In [80]:
df_test_size.Class.value_counts()

0    1724
1     276
Name: Class, dtype: int64

In [62]:
df_fit.head()

,Sentence,Class,Aspect
Index,,,
2010,no fading and everything seems to hold its sha...,0,Fit
2011,i do recommend.,0,Fit
2012,very comfortable.,0,Fit
2013,hold everything in place as expected during al...,0,Fit
2014,i own 3 of these and love them.,0,Fit


# Aspect Fit

In [65]:
import csv
train_df_fit = df_fit.copy()
test_df_fit = df_test_fit.copy()

x_train_fit = train_df_fit[['Sentence', 'Aspect']].values
y_train_fit = train_df_fit['Class'].values
x_test_fit = test_df_fit[['Sentence', 'Aspect']].values
y_test_fit = test_df_fit['Class'].values

# IMPORTANT: data format for sentence pair classification is list of tuples of form (str, str)
x_train_fit = list(map(tuple, x_train_fit))
x_test_fit = list(map(tuple, x_test_fit))

#x_train_fit

In [74]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-uncased'
t_fit = text.Transformer(MODEL_NAME, maxlen=128, class_names=['not_fit', 'fit'])
trn = t_fit.preprocess_train(x_train_fit, y_train_fit)
val = t_fit.preprocess_test(x_test_fit, y_test_fit)
model_fit = t_fit.get_classifier()
learner_fit = ktrain.get_learner(model_fit, train_data=trn, batch_size=8) # lower bs if OOM occurs
learner_fit.fit_onecycle(5e-5, 4)

preprocessing train...
language: en
sentence pairs detected


Is Multi-Label? False
preprocessing test...
language: en
sentence pairs detected




begin training using onecycle policy with max lr of 5e-05...
Epoch 1/4
1004/1004 [==============================] - 328s 327ms/step - loss: 0.1910 - accuracy: 0.9307
Epoch 2/4
1004/1004 [==============================] - 327s 326ms/step - loss: 0.1246 - accuracy: 0.9568
Epoch 3/4
1004/1004 [==============================] - 327s 326ms/step - loss: 0.1136 - accuracy: 0.9592
Epoch 4/4
1004/1004 [==============================] - 327s 326ms/step - loss: 0.0818 - accuracy: 0.9688


In [75]:
learner_fit.validate(val_data=val)

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1721
           1       0.88      0.97      0.92       279

    accuracy                           0.98      2000
   macro avg       0.94      0.97      0.95      2000
weighted avg       0.98      0.98      0.98      2000



array([[1684,   37],
       [   9,  270]])

In [90]:
predictor_fit = ktrain.get_predictor(learner_fit.model, t_fit)
predictor_fit.save('/content/drive/My Drive/Research/Data/fit_model')

In [84]:
x_test_fit[0]

('my son loved his sweater but it was a bit too fitted for his liking.. looks very nice and he definitely wears it.',
 'Fit')

In [ ]:
x_test_fit

In [92]:
fit = ktrain.load_predictor('/content/drive/My Drive/Research/Data/fit_model')
fit.predict([x_test_fit[0]])

['fit']

In [95]:
fit_predictions = fit.predict(x_test_fit)

In [96]:
df_fit_final = df_test_fit.copy()
df_fit_final['predicted'] = fit_predictions
df_fit_final['predicted'] = df_fit_final['predicted'].map({'fit':1,'not_fit':0})
df_fit_final.head()

,Sentence,Class,Aspect,predicted
Index,,,,
0,my son loved his sweater but it was a bit too ...,1,Fit,1
1,i would suggest ordering a size up,0,Fit,0
2,cannot go wrong with nike products for the mos...,0,Fit,0
3,fit and finish is excellent.,1,Fit,1
4,also surprised how water resistant the pullove...,0,Fit,0


In [97]:
print('F1 score - Fit :',f1_score(df_fit_final['Class'], df_fit_final['predicted']))

F1 score - Fit : 0.9215017064846416


In [103]:
print(classification_report(df_fit_final['Class'], df_fit_final['predicted']))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1721
           1       0.88      0.97      0.92       279

    accuracy                           0.98      2000
   macro avg       0.94      0.97      0.95      2000
weighted avg       0.98      0.98      0.98      2000



In [104]:
print(confusion_matrix(df_fit_final['Class'], df_fit_final['predicted']))

[[1684   37]
 [   9  270]]


# Aspect Size

In [105]:
import csv
train_df_size = df_size.copy()
test_df_size = df_test_size.copy()

x_train_size = train_df_size[['Sentence', 'Aspect']].values
y_train_size = train_df_size['Class'].values
x_test_size = test_df_size[['Sentence', 'Aspect']].values
y_test_size = test_df_size['Class'].values

# IMPORTANT: data format for sentence pair classification is list of tuples of form (str, str)
x_train_size = list(map(tuple, x_train_size))
x_test_size = list(map(tuple, x_test_size))


In [128]:
t_size = text.Transformer(MODEL_NAME, maxlen=128, class_names=['not_size', 'size'])
trn_size = t_size.preprocess_train(x_train_size, y_train_size)
val_size = t_size.preprocess_test(x_test_size, y_test_size)
model_size = t_size.get_classifier()
learner_size = ktrain.get_learner(model_size, train_data=trn_size, batch_size=8) # lower bs if OOM occurs
learner_size.fit_onecycle(5e-5, 4)

preprocessing train...
language: en
sentence pairs detected


Is Multi-Label? False
preprocessing test...
language: en
sentence pairs detected


ValueError: ignored

In [ ]:
learner_size.validate(val_data=val_size)

In [ ]:
predictor_size = ktrain.get_predictor(learner_size.model, t_size)
predictor_size.save('/content/drive/My Drive/Research/Data/size_model')

In [107]:
size = ktrain.load_predictor('/content/drive/My Drive/Research/Data/size_model')
size.predict([x_test_size[0]])

['not_size']

In [108]:
size_predictions = size.predict(x_test_size)

In [109]:
df_size_final = df_test_size.copy()
df_size_final['predicted'] = size_predictions
df_size_final['predicted'] = df_size_final['predicted'].map({'size':1,'not_size':0})
df_size_final.head()

,Sentence,Class,Aspect,predicted
Index,,,,
0,my son loved his sweater but it was a bit too ...,0,Size,0
1,i would suggest ordering a size up,1,Size,1
2,cannot go wrong with nike products for the mos...,0,Size,0
3,fit and finish is excellent.,0,Size,0
4,also surprised how water resistant the pullove...,0,Size,0


In [110]:
print('F1 score - Size :',f1_score(df_size_final['Class'], df_size_final['predicted']))

F1 score - Size : 0.8247011952191236


# Aspect Material

In [123]:
train_df_material = df_material.copy()
test_df_material = df_test_material.copy()
#print(train_df_material)
x_train_material = train_df_material[['Sentence', 'Aspect']].values
y_train_material = train_df_material['Class'].values
x_test_material = test_df_material[['Sentence', 'Aspect']].values
y_test_material = test_df_material['Class'].values

# IMPORTANT: data format for sentence pair classification is list of tuples of form (str, str)
x_train_material = list(map(tuple, x_train_material))
x_test_material = list(map(tuple, x_test_material))

In [127]:
MODEL_NAME = 'bert-base-uncased'
t_material = text.Transformer(MODEL_NAME, maxlen=128, class_names=['not_material', 'material'])
trn_material = t_material.preprocess_train(x_train_material, y_train_material)
val_material = t_material.preprocess_test(x_test_material, y_test_material)
model_material = t_material.get_classifier()
learner_material = ktrain.get_learner(model_material, train_data=trn_material, batch_size=5) # lower bs if OOM occurs
learner_material.fit_onecycle(5e-5, 4)

preprocessing train...
language: en
sentence pairs detected


Is Multi-Label? False
preprocessing test...
language: en
sentence pairs detected


ValueError: ignored